In [2]:
import json
import re

from pyspark.sql.functions import (regexp_extract, regexp_replace, col, lower,
                                   when, isnull, coalesce, udf, StringType)
import pandas as pd

In [3]:
path = 'test'

In [4]:
# loads multiple text files
rdd = sc.textFile(path)

In [5]:
def parse(x):
    data = json.loads(x)
    key = list(data.keys())[0]
    dic = data[key]
    
    if len(dic['author']) > 0:
        dic['author'] = dic['author'][0]
    else:
        dic['author'] = None
        
    if len(dic['language']) > 0:
        dic['language'] = dic['language'][0]
    else:
        dic['language'] = None
        
    if len(dic['subject']) > 0:
        dic['subject'] = dic['subject'][0]
    else:
        dic['subject'] = None
        
    if len(dic['title']) > 0:
        dic['title'] = dic['title'][0]
    else:
        dic['title'] = None
    
    print(str(dic['txt'][0]))
    dic['release_date'] = re.match(r"(?<=Release Date:).*(?=(\[E))", str(dic['txt'][0]))
    if dic['release_date'] == None:
        dic['release_date'] = str(dic['txt'][0])
        
    
        
    return dic

In [6]:
rdd = rdd.map(parse)

In [7]:
# perform work on rdd
df = rdd.toDF()

/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/session.py:356: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [20]:
# pull out release date
df = df.withColumn('release_date', regexp_extract(col('txt'), r"(?<=Release Date:)(.*)(?=(\[E))", 1))

# strip out newline characters
df = df = df.withColumn('txt', regexp_replace(col('txt'), "[\n]", " "))

# lowercase the txt
df = df.withColumn('txt', lower(col('txt')))

# use regex to remove bad parts of the text gutenberg adds at the end
df = df.withColumn('end', regexp_extract(col('txt'), r"(.*)(?=end of the project gutenberg)", 1))
df = df.withColumn('end', when(df.end == "",
                               regexp_extract(col('txt'), r"(.*)(?=end of this project gutenberg)", 1)).otherwise(df.end))

# delete end column
df = df.withColumn('txt', col('end'))
df = df.drop('end')

# use regex to remove bad parts of the start of gutenberg texts
df = df.withColumn('start', regexp_extract(col('txt'), r"(?<=start of this project gutenberg)(.*)", 1))
df = df.withColumn('start', when(df.start == "",
                               regexp_extract(col('txt'), r"(?<=start of the project gutenberg)(.*)", 1)).otherwise(df.start))


In [ ]:
# perform operations
df_p = df.toPandas()

In [ ]:
df_p['txt'][0][:8000]

In [13]:
df = df.withColumn('txt', col('end'))